In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json
import psycopg2
from sodapy import Socrata
import re
from transform_dag import transformations_api

In [2]:
def read_db_water():
    with open('db_config.json') as file:
        db_config = json.load(file)

    engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:5433/{db_config["dbname"]}')

    water = pd.read_sql('SELECT * FROM water_table', engine)
    
    
    return water

In [4]:
water =read_db_water()
print(water.head())

          Año NombreDepartamento  Div_dpto NombreMunicipio  Divi_muni  \
0  2010-01-01            Bolívar        13        El Guamo      13248   
1  2010-01-01            Bolívar        13        El Guamo      13248   
2  2010-01-01            Bolívar        13        El Guamo      13248   
3  2010-01-01            Bolívar        13        El Guamo      13248   
4  2010-01-01            Bolívar        13        El Guamo      13248   

   IrcaMinimo  IrcaMaximo  IrcaPromedio NombreParametroAnalisis2  \
0         0.0       100.0         37.32        Alcanilidad Total   
1         0.0       100.0         37.32                 Aluminio   
2         0.0       100.0         37.32                 Arsénico   
3         0.0       100.0         37.32                   Cadmio   
4         0.0       100.0         37.32                   Calcio   

   MuestrasEvaluadas  MuestrasTratadas  MuestrasSinTratar  \
0                 67                67                  0   
1                 67          

In [3]:
from transform_dag import transformations_water

def transform_water():
    
    water = read_db_water()
    
    water = transformations_water(water)
    
    print(water.info())
    print(water.head())

# Llamar a la función principal para procesar los datos
transform_water()



2024-05-18 07:22:55,751 - INFO - Starting transformations on water data.
2024-05-18 07:22:55,864 - INFO - Dates converted successfully.
2024-05-18 07:22:56,367 - INFO - normalize text colums water succesfully
2024-05-18 07:22:56,401 - INFO - Scaled numerical columns.
2024-05-18 07:22:56,456 - INFO - Filtered top influential parameters.
2024-05-18 07:22:56,555 - INFO - Classified IRCA values into categories.
2024-05-18 07:22:59,690 - INFO - Categorized treatment data.
2024-05-18 07:23:04,050 - INFO - Critical Proportion.
2024-05-18 07:23:04,095 - INFO - Dropped unnecessary columns.
2024-05-18 07:23:04,245 - INFO - renombrar columnass water succesfully


KeyError: 'NombreDepartamento'

In [4]:
def extract_api(endpoint):
    try:
        # Inicializa el cliente de Socrata sin token de aplicación
        client = Socrata("www.datos.gov.co", None)

        # Obtiene los datos desde el endpoint especificado
        results = client.get(endpoint, limit=2000)

        # Convierte los resultados en un DataFrame
        api = pd.DataFrame.from_records(results)

        return api
    except Exception as e:
        print(f"Se produjo un error: {e}")

In [5]:
api = extract_api("tcwu-r53g")
print(api.head())

2024-05-17 21:46:04,100 - WARNING - Requests made without an app_token will be subject to strict throttling limits.


  fecha_terminacion_proyecto           fecha_de_corte  \
0    2018-12-31T00:00:00.000  2024-04-04T00:00:00.000   
1    2018-10-16T00:00:00.000  2024-04-04T00:00:00.000   
2    2018-10-08T00:00:00.000  2024-04-04T00:00:00.000   
3    2019-06-01T00:00:00.000  2024-04-04T00:00:00.000   
4    2019-02-12T00:00:00.000  2024-04-04T00:00:00.000   

  c_digo_divipola_departamento departamento c_digo_divipola_municipio  \
0                           18      CAQUETA       18029, 18205, 18610   
1                           15       BOYACA                     15693   
2                           17       CALDAS                     17050   
3                           17       CALDAS                     17446   
4                           05    ANTIOQUIA                      5250   

                                           municipio  \
0  Albania(CAQ), Curillo(CAQ), San Jose De Fragua...   
1                         Santa Rosa De Viterbo(BOY)   
2                                      Aranzazu(CA

In [11]:

def transform_api():
    
    pd.set_option('display.max_columns', None)  # Ninguna limitación en el número de columnas a mostrar
    pd.set_option('display.max_rows', None)  # Ninguna limitación en el número de filas a mostrar
    
    api = extract_api("tcwu-r53g")
    
    api = transformations_api(api)
    
    print(api.head())

transform_api()

2024-05-17 22:56:02,764 - WARNING - Requests made without an app_token will be subject to strict throttling limits.
2024-05-17 22:56:03,710 - INFO - Starting transformations on API data.
2024-05-17 22:56:03,712 - INFO - Elimination of parentheses within municipalities successfully.
2024-05-17 22:56:03,719 - INFO - Separates records with multiple municipalities into individual rows successfully.
2024-05-17 22:56:03,720 - INFO - Elimination of extra spaces and capitalization of each municipality name successful.
2024-05-17 22:56:03,728 - INFO - Dates converted successfully.
2024-05-17 22:56:03,740 - INFO - Text columns normalized successfully.
2024-05-17 22:56:03,741 - INFO - Number of municipalities computed successfully.
2024-05-17 22:56:03,745 - INFO - Regions mapped successfully.
2024-05-17 22:56:03,747 - INFO - Project financing calculated successfully.
2024-05-17 22:56:03,755 - INFO - Project duration calculated successfully.
2024-05-17 22:56:03,759 - INFO - Unnecessary columns dro

  fecha_terminacion_proyecto c_digo_divipola_departamento departamento  \
0                 2018-12-31                           18      CAQUETA   
0                 2018-12-31                           18      CAQUETA   
0                 2018-12-31                           18      CAQUETA   
1                 2018-10-16                           15       BOYACA   
2                 2018-10-08                           17       CALDAS   

  c_digo_divipola_municipio              municipio  \
0       18029, 18205, 18610                albania   
0       18029, 18205, 18610                curillo   
0       18029, 18205, 18610     san jose de fragua   
1                     15693  santa rosa de viterbo   
2                     17050               aranzazu   

                                           indicador  \
0       nueva población beneficiada acueducto urbano   
0       nueva población beneficiada acueducto urbano   
0       nueva población beneficiada acueducto urbano   
1     

In [32]:
api_done_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/ETL-Proyect/Main_Aqua_Quality/API/DATA/api_done.csv')

water_cleaned_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/water_cleaned.csv')

# Convertir columnas de fechas a formato datetime
api_done_df['fecha_terminacion_proyecto'] = pd.to_datetime(api_done_df['fecha_terminacion_proyecto'])
water_cleaned_df['año'] = pd.to_datetime(water_cleaned_df['año'])



# Filtrar el dataset de proyectos para incluir solo los años 2018 y 2019
api_done_filtered_df = api_done_df[api_done_df['fecha_terminacion_proyecto'].dt.year.isin([2018, 2019])]





# Renombrar la columna 'municipio' a 'nombre_municipio' y 'fecha_terminacion_proyecto' a 'fecha_proyecto'
api_done_filtered_df = api_done_filtered_df.rename(columns={'municipio': 'nombre_municipio', 'fecha_terminacion_proyecto': 'fecha_proyecto'})

# Función para limpiar y dividir nombres de municipios en el dataset de proyectos
def clean_and_split_municipios(municipios):
    # Eliminar texto entre paréntesis y caracteres especiales
    cleaned = re.sub(r'\(.*?\)', '', municipios).replace(',', '').strip().lower()
    # Dividir si hay múltiples municipios
    return [m.strip().title() for m in cleaned.split()]

# Aplicar la limpieza y división a los nombres de municipios en el dataset de proyectos
api_done_filtered_df['nombre_municipio'] = api_done_filtered_df['nombre_municipio'].apply(lambda x: ', '.join(clean_and_split_municipios(x)))
api_done_filtered_df = api_done_filtered_df.assign(nombre_municipio=api_done_filtered_df['nombre_municipio'].str.split(', ')).explode('nombre_municipio').reset_index(drop=True)

# Función para limpiar nombres de municipios en el dataset de calidad del agua
def clean_municipio_name_simple(name):
    return name.strip().title()

# Aplicar la limpieza a los nombres de municipios en el dataset de calidad del agua
water_cleaned_df['nombre_municipio'] = water_cleaned_df['nombre_municipio'].apply(clean_municipio_name_simple)




# Filtrar el dataset de calidad del agua para incluir solo las fechas de medición a partir de 2018
water_cleaned_filtered_df = water_cleaned_df[water_cleaned_df['año'].dt.year >= 2018]

# Realizar el merge de tipo "asof" usando los nombres de municipios normalizados
merged_df = pd.merge_asof(
    water_cleaned_filtered_df.sort_values('año'),
    api_done_filtered_df.sort_values('fecha_proyecto'),
    by='nombre_municipio',
    left_on='año',
    right_on='fecha_proyecto',
    direction='backward'
)

# Eliminar columnas redundantes excepto "fecha_de_corte"
columns_to_drop = ['c_digo_divipola_departamento', 'departamento', 'c_digo_divipola_municipio', 'fecha_de_corte', 'num_municipios']
merged_df = merged_df.drop(columns=columns_to_drop)

# Llenar las entradas faltantes
merged_df.fillna({
    'indicador': 'Ausencia de proyecto',
    'nombre_proyecto': 'Ausencia de proyecto',
    'origen': 'Ausencia de proyecto',
    'estado_seguimiento': 'Ausencia de proyecto',
    'región': 'Ausencia de proyecto',
    'aporte_nacion': -1,
    'contrapartida': -1,
    'total_financiamiento': -1,
    'fecha_proyecto': -1,
    'duracion_proyecto_dias': -1
}, inplace=True)

# Resultados
print(f"Municipios con proyectos: {merged_df[merged_df['nombre_proyecto'] != 'Ausencia de proyecto']['nombre_municipio'].nunique()}")
print(f"Número total de filas en el DataFrame resultante: {merged_df.shape[0]}")
print(f"Años disponibles en el DataFrame resultante: {merged_df['año'].dt.year.unique()}")
merged_df.to_csv('merged_water.csv', index=False)



Municipios con proyectos: 19
Número total de filas en el DataFrame resultante: 32355
Años disponibles en el DataFrame resultante: [2018 2019]
